In [1]:
import pandas as pd
import os
import numpy as np

## Read Data

In [2]:
path = '.../Data.csv'
data=pd.read_csv(path)
data

,GRAVYREGISTRATIONID,GEOHASHLATITUDE,GEOHASHLONGITUDE,GEOHASH9,TIMESTAMP_EPOCH_MILLISECOND,TIMEZONE,FORENSICFLAG,time,LAT_EPSG_3857,LON_EPSG_3857
0,00056b3f-b4af-320f-bde2-70a95624a2b3,38.325870,-122.489039,9qbfh9n0q,1571184720000,America/Los_Angeles,32768,2019-10-16 00:12:00,4.625563e+06,-1.363542e+07
1,00056b3f-b4af-320f-bde2-70a95624a2b3,38.325870,-122.488996,9qbfh9n0r,1571184720000,America/Los_Angeles,32768,2019-10-16 00:12:00,4.625563e+06,-1.363541e+07
2,00056b3f-b4af-320f-bde2-70a95624a2b3,38.325870,-122.488739,9qbfh9n2m,1571184720000,America/Los_Angeles,0,2019-10-16 00:12:00,4.625563e+06,-1.363538e+07
3,00056b3f-b4af-320f-bde2-70a95624a2b3,38.325870,-122.489039,9qbfh9n0q,1571184720000,America/Los_Angeles,32768,2019-10-16 00:12:00,4.625563e+06,-1.363542e+07
4,00056b3f-b4af-320f-bde2-70a95624a2b3,38.325870,-122.489039,9qbfh9n0q,1571184720000,America/Los_Angeles,32768,2019-10-16 00:12:00,4.625563e+06,-1.363542e+07
...,...,...,...,...,...,...,...,...,...,...
10213100,fffe49e5-6d85-37d3-b674-6f75445477ed,38.500750,-122.468740,9qbgj0ugp,1571720700000,America/Los_Angeles,4194304,2019-10-22 05:05:00,4.650409e+06,-1.363316e+07
10213101,fffe49e5-6d85-37d3-b674-6f75445477ed,38.500621,-122.468698,9qbgj0v42,1571720700000,America/Los_Angeles,4194304,2019-10-22 05:05:00,4.650390e+06,-1.363315e+07
10213102,fffe49e5-6d85-37d3-b674-6f75445477ed,38.500664,-122.468783,9qbgj0ufw,1571720700000,America/Los_Angeles,4194304,2019-10-22 05:05:00,4.650396e+06,-1.363316e+07
10213103,fffe49e5-6d85-37d3-b674-6f75445477ed,38.500578,-122.468826,9qbgj0ufj,1571720700000,America/Los_Angeles,4194304,2019-10-22 05:05:00,4.650384e+06,-1.363317e+07


In [3]:
ID_list=data['GRAVYREGISTRATIONID'].unique()
len(ID_list)

4041

## Home Inference Function

by identifying hottest spot during night

In [4]:
def home_location(ndf,s_hour=22,t_hour=6):
    '''
    ndf: gps data
    s_hour: define start time of night, defult = 22
    t_hour: define end time of night, defult = 6
    '''
    ndf['time']=pd.to_datetime(ndf['time'],utc=True)
    ndf['hour']=pd.Series(ndf['time']).dt.tz_convert('US/Pacific').dt.hour
    ndf['LAT_Grid']=np.round(ndf['LAT_EPSG_3857']/20)*20  # Grid size = 20m
    ndf['LON_Grid']=np.round(ndf['LON_EPSG_3857']/20)*20
    id=ndf.iloc[0,0]
    ndf=ndf[(ndf['hour']>=s_hour)|(ndf['hour']<t_hour)]
    if len(ndf)==0:  # if no signal during the night, return NaN
        return np.nan,np.nan
    home=ndf.groupby(['LAT_Grid','LON_Grid']).count()['GEOHASH9'].idxmax()
    return home[0],home[1]

In [5]:
home=pd.DataFrame(columns=['ID','LAT','LON']) # initial a DataFrame to save home location

for i in ID_list[0:5]: 
    df=data[data['GRAVYREGISTRATIONID']==i].sort_values('TIMESTAMP_EPOCH_MILLISECOND',axis=0,ascending=True) # extract records
    h_lat,h_lon=home_location(df) # calculate home location
    home=home.append(pd.DataFrame([[i,h_lat,h_lon]],columns=['ID','LAT','LON'])) # append the new row 

In [6]:
home

,ID,LAT,LON
0,00056b3f-b4af-320f-bde2-70a95624a2b3,4627480.0,-13637720.0
0,00230920-5dc9-388f-93bf-789cbed249b8,4614500.0,-13649900.0
0,00364719-aa13-3eef-841d-9f319954a068,4645500.0,-13654600.0
0,003652a2-9b4f-335f-ade9-8519a75ff3f6,4634080.0,-13680420.0
0,0038f14e-4a3e-3d50-93c7-4f6bf8c0434e,4613740.0,-13646560.0
